## Cloning the Repository

In [1]:
# All vid gen techniques are described in
# https://huggingface.co/docs/diffusers/main/en/using-diffusers/text-img2vid
# https://huggingface.co/docs/diffusers/v0.31.0/using-diffusers/text-img2vid

# clone our repo and install requirements in colab
%cd /content/
!rm -rf LSVD
!git clone -b Inference https://github.com/davidg-h/LSVD.git
%cd LSVD
%pip install -r requirements.txt
!apt install ffmpeg
!git pull origin Inference

/content
Cloning into 'LSVD'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 178 (delta 72), reused 136 (delta 45), pack-reused 0 (from 0)
Receiving objects: 100% (178/178), 6.62 MiB | 3.65 MiB/s, done.
Resolving deltas: 100% (72/72), done.
/content/LSVD
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 803.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.4/883.4 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB

## Setting variables for audio/video generation

In [1]:
fps = 7
num_frames = 49 #49 / 7fps = ~6s
decode_chunk_size=2
prompt = "A happy woman walking down an aile in a convenience store, because she finally bought her favorite chocolate ice cream flavor, FHD"
negt = "bad quality, worse quality, low resolution"

## Text to Audio Generation

In [3]:
from transformers import pipeline, AutoProcessor, MusicgenForConditionalGeneration
import scipy

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

# Input from user is stored into userInput
userInput = prompt

# analysis stores the classified user input for with the different confidence level of the emotions detected
analysis = classifier(userInput)
filtered_analysis = sorted([item for item in analysis[0] if item['score'] > 0.4], key=lambda x: x['score'], reverse=True)

labels_string = "mood: " + ", ".join([item['label'] for item in filtered_analysis])
print(labels_string)

edited_user_input = "\n".join([userInput, labels_string])
print(f"Edited uder input: {edited_user_input}")

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=[edited_user_input],
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs, max_new_tokens=400)

# Output the final audio into a .wav file
sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].numpy())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 5

generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

`torch.nn.functional.scaled_dot_product_attention` does not support having an empty attention mask. Falling back to the manual attention implementation. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.Note that this probably happens because `guidance_scale>1` or because you used `get_unconditional_inputs`. See https://github.com/huggingface/transformers/issues/31189 for more information.


In [4]:
prompt = edited_user_input
prompt

'A happy woman walking down an aile in a convenience store, because she finally bought her favorite chocolate ice cream flavor.\nmood: joy'

In [5]:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].numpy(), rate=sampling_rate)

## Text to Image Generation

In [6]:
# https://huggingface.co/tasks/text-to-image
# https://huggingface.co/black-forest-labs/FLUX.1-dev
# https://huggingface.co/docs/diffusers/using-diffusers/conditional_image_generation
# https://huggingface.co/blog/if
# https://huggingface.co/docs/diffusers/en/api/pipelines/stable_diffusion/text2img
# https://huggingface.co/docs/diffusers/v0.16.0/en/api/pipelines/stable_diffusion/text2img

import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

model_id = "stabilityai/stable-diffusion-2"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.enable_model_cpu_offload()

image = pipe(prompt).images[0]

image.save("reference.png")

# cleanup
pipe.to("cpu")
del pipe
torch.cuda.empty_cache()

scheduler/scheduler_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

text_encoder/config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

## Image to Stable Video Diffusion

In [5]:
# Img to Video Stable Video Diffusion

import torch
from diffusers import StableVideoDiffusionPipeline, DiffusionPipeline
from diffusers.utils import load_image, export_to_video

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
)

# Optimize
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking()

image = load_image(
    "/content/LSVD/reference.png"
    #"https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/svd/rocket.png"
    ) # hf example
image = image.resize((1024, 576))

generator = torch.manual_seed(42)
try:
  frames = pipe(image,
                num_frames=num_frames,
                decode_chunk_size=8,
                generator=generator
                ).frames[0]
  export_to_video(frames, "svd.mp4", fps=fps)
except Exception as e:
  raise Exception(e)
finally:
  # cleanup
  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()

# Worked in colab (stable; quality is mediocre --> movement aliasing)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Exception: `prompt` has to be of type `str` or `list` but is <class 'PIL.Image.Image'>

## Combining the Audio and Video

In [ ]:
import os
import subprocess

def install_ffmpeg():
    try:
        # Update the package list
        print("Updating package list...")
        subprocess.run(["sudo", "apt", "update"], check=True)

        # Install FFmpeg
        print("Installing FFmpeg...")
        subprocess.run(["sudo", "apt", "install", "-y", "ffmpeg"], check=True)

        print("FFmpeg installed successfully!")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while installing FFmpeg: {e}")

def fuse_video_audio(video_file, audio_file, output_file):
    try:
        # Use FFmpeg to merge video and audio
        print(f"Merging {video_file} and {audio_file} into {output_file}...")
        subprocess.run(["ffmpeg", "-i", video_file, "-i", audio_file, "-c:v", "copy", "-c:a", "aac", "-strict", "experimental", output_file], check=True)
        print(f"Successfully merged into {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while merging video and audio: {e}")


# Example usage
fuse_video_audio("svd.mp4", "musicgen_out.wav", "advertisement.mp4")